# Compile and Summarise Derivative Cover Check Sheets

In [1]:
# Create a time difference function

import datetime, time

def timediff(start, end, decimals = 1):
    if int((end - start)/3600) > 0: # non-zero hours
        return str(int((end - start)/3600)) + 'hr ' + str(int((end - start)/60)) +'min ' + str(round((end - start) % 60,decimals)) + 'sec'
    elif int((end - start)/60) > 0: # non-zero hours and minutes
        return str(int((end - start)/60)) +'min ' + str(round((end - start) % 60, decimals)) + 'sec'
    else:
        return str(round((end - start) % 60,decimals)) + 'sec'

In [2]:
# Import libraries

start_time          = time.time()
start_time_overlord = start_time
print('Importing libraries ...')

import pandas as pd
import xlwings as xw
from pathlib import Path
import os, shutil
import win32com.client as win32 # library to convert xls to xlsx

from tqdm import tqdm, notebook # notebook version of tqdm
#" ...any time you see a loop somewhere in your code in you can simply wrap it in either tdqm() or tqdm_notebook() in Jupyter" 

print(f'Importing libraries completed: {timediff(start_time, time.time())}', '\n')

Importing libraries ...
Importing libraries completed: 1.1sec


In [3]:
# Start Excel

start_time = time.time()
print('Starting Excel ...')

# start Excel - if Excel does not open, close any Excel application already open, else restart the pc
excel = win32.gencache.EnsureDispatch('Excel.Application')

print(f'Starting Excel completed: {timediff(start_time, time.time())}', '\n')

Starting Excel ...
Starting Excel completed: 0.0sec


In [4]:
# Set location paths for the sheets to be used

start_time = time.time()
print('Setting up paths ...')

pthEXPORTS    = r'\\PIM-CPT-FS.prescient.local\PIM-Documents$\Investment Operations\GRC\Compliance\Derivative Cover'
pthDerv       = r'\\PIM-CPT-FS.prescient.local\PIM-Documents$\Investment Operations\GRC\Compliance\Daily'
pthFundCodes  = r'\\PIM-CPT-FS.prescient.local\PIM-Documents$\Investment Operations\GRC\Compliance\Daily\2A - Fund Codes, Breach Register.xlsx'
pthMandates   = r'\\PIM-CPT-FS.prescient.local\PIM-Documents$\Investment Operations\GRC\Compliance\Client Mandates' # path to mandate summaries
fPARN         = r'C:\Users\hilton.netta\Downloads\Portfolio Analytics Report - New.xls'
fDE           = r'C:\Users\hilton.netta\Downloads\Derivative Exposure.xls'
pthLOCAL      = r'C:\Users\hilton.netta\Documents\DervFiles'     # path to were the files will be saved locally temporarily
pthOverdrafts = r'\\pim-cpt-statpro\SPC_602\Client\Profiles\PIM\overdrafts'
pthDx         = r'C:\Users\hilton.netta\Downloads'

print(f'Setting up paths completed: {timediff(start_time, time.time())}', '\n')

Setting up paths ...
Setting up paths completed: 0.0sec


In [5]:
# Get report date and selected summary sheet option

start_time = time.time()
print('Getting the reporting date ...')

import xlwings as xw
wb      = xw.Book(r'P:\Investment Operations\GRC\Compliance\Daily\py_reports.xlsm') # open calc workbook as an object
rptDate = wb.sheets['dervs'].range('B1').value.strftime("%d%b%Y")                   # get report date in ddMmmYYYY format
summ_yn = wb.sheets['dervs'].range('B2').value
wb.close()

print(f' Reporting date: {rptDate}')
print(f'Getting the reporting date: {timediff(start_time, time.time())}', '\n')

Getting the reporting date ...
 Reporting date: 21Sep2023
Getting the reporting date: 2.3sec


In [6]:
# Get the names of the latest files just saved in the Downloads folder

start_time = time.time()
print('Getting names of downloaded holdings and derivative files ...')

pth  =  r'C:\Users\hilton.netta\Downloads' # where downloaded sheets from Eagle will be saved 
parN = 'Portfolio Analytics Report - New'  # name of the downloaded holdings sheet from Eagle
der  = 'Derivative Exposure'               # name of the downloaded derivative data sheet from Eagle

#https://stackoverflow.com/questions/64255610/read-latest-file-with-a-filename-contains-string-in-python
#https://stackoverflow.com/questions/9816816/get-absolute-paths-of-all-files-in-a-directory
fls     = os.listdir(r'C:\Users\hilton.netta\Downloads')
parN_nm = max([os.path.abspath(os.path.join(pth, fl)) for fl in fls if parN in fl], key = os.path.getmtime)
der_nm  = max([os.path.abspath(os.path.join(pth, fl)) for fl in fls if der  in fl], key = os.path.getmtime)
print('', parN_nm, '\n', der_nm)

print(f'Getting names of downloaded holdings and derivative files completed: {timediff(start_time, time.time())}', '\n')

Getting names of downloaded holdings and derivative files ...
 C:\Users\hilton.netta\Downloads\Portfolio Analytics Report - New(29).xls 
 C:\Users\hilton.netta\Downloads\Derivative Exposure.xlsx
Getting names of downloaded holdings and derivative files completed: 0.0sec


In [7]:
# Convert .xls to .xlsx using MS Office for the Eagle holdings and derivative delta files

start_time = time.time()
print('Converting xls files to xlsx ...')

excel.DisplayAlerts = False # suppress the warning dialogue

wb = excel.Workbooks.Open(parN_nm)
wb.SaveAs(os.path.join(pth, 'p.xlsx'), FileFormat = 51)     #FileFormat = 51 is for .xlsx extension
wb.Close()                               #FileFormat = 56 is for .xls extension

wb = excel.Workbooks.Open(der_nm)
wb.SaveAs(os.path.join(pth, 'd.xlsx'), FileFormat = 51)     #FileFormat = 51 is for .xlsx extension
wb.Close()       

excel.DisplayAlerts = True # unsuppress Excel warning dialogue

#excel.Application.Quit()

print(f'Converting xls files to xlsx completed: {timediff(start_time, time.time())}', '\n')

Converting xls files to xlsx ...
Converting xls files to xlsx completed: 0.5sec


In [8]:
# Create dataframes of the holdings and deltas spreadsheets

start_time = time.time()
print('Creating dataframes from fund holdings and derivative data ...')


wbD = pd.read_excel(open(pthDx + '/d.xlsx','rb'), sheet_name = 'Derivative Exposure')
wbH = pd.read_excel(open(pthDx + '/p.xlsx','rb'), sheet_name = 'Portfolio Analytics Report - Ne')

# get the report date and list of fund names
# rptDate = wbH.iloc[0, 35].strftime("%d%b%Y") # 1st row 36th column contains the report date
fnames  = wbH['Entity Name'].unique() # list of the fund names in the holdings file, resulting type = numpy.ndarray

print(f'Creating dataframes from fund holdings and derivative data completed: {timediff(start_time, time.time())}', '\n')

Creating dataframes from fund holdings and derivative data ...
Creating dataframes from fund holdings and derivative data completed: 0.4sec


In [12]:
# Change the '% of Total Market Value" column

start_time = time.time()
print('Calculating and saving fund Total Market Value percentages ...')

# change the '% of Total Market Value" column (N) to the fund-specific % based on 'Sum of Market Value Income' column (M)
navs = wbH.groupby('Entity ID')['Sum of Market Value Income'].sum() # (column N) this has type 'pandas.core.series.Series'
nav  = navs.to_dict() # nav series changed to dictionary to make it lookupable

# recalc the '% of Total Market Value' column per fund and ...
newTMV = []
for i, row in notebook.tqdm(wbH.iterrows()):
    if nav[row['Entity ID']] == 0:
        fndpct = 100
    else:
        fndpct = row['Sum of Market Value Income'] / nav[row['Entity ID']] * 100
    newTMV.append(fndpct)
    
# ... replace the '% of Total Market Value' with the values in the new list
wbH['% of Total Market Value'] = newTMV # wbH['% of Total Market Value'].sum(), check, should equal number of funds

print(f'Calculating and saving fund Total Market Value percentages completed: {timediff(start_time, time.time())}', '\n')

Calculating and saving fund Total Market Value percentages ...


0it [00:00, ?it/s]

Calculating and saving fund Total Market Value percentages completed: 0.0sec


In [13]:
# Recalculate % of Total Market Value' column

start_time = time.time()
print('Recalculating Current Exposure percentage column ...')

# recalc the '% of Total Market Value' column per fund and ...
newCEp = [] # new Current Exposure % column
for i, row in notebook.tqdm(wbH.iterrows()):
    if nav[row['Entity ID']] == 0:
        currentexposurepct = 1
    else:
        currentexposurepct = row['Current Exposure'] / nav[row['Entity ID']] * 100
    newCEp.append(currentexposurepct )
    
# ... replace the '% of Total Market Value' with the values in the new list
wbH['Current Exposure %'] = newCEp # wbH['% of Total Market Value'].sum(), check, should equal number of funds

print(f'Recalculating Current Exposure percentage column completed: {timediff(start_time, time.time())}', '\n')

Recalculating Current Exposure percentage column ...


0it [00:00, ?it/s]

Recalculating Current Exposure percentage column completed: 0.0sec


In [14]:
# Create the individual derivative checker files locally after opening the derv calc workbook

start_time = time.time()
print('Creating individual fund derivative calculation files ...')

# create a lookup table for fund UT status and investment team
twoA = pd.read_excel(pthFundCodes, sheet_name = 'Funds', usecols = 'A, D:E')

# open the derv template and assign holdings and data sheets
wb   = xw.Book(pthDerv + r'\derv.xlsx') # open the derv calc workbook as an object
shtH = wb.sheets['Data'   ]             # assign sheet containing holdings as obtained from Eagle
shtD = wb.sheets['Deltas' ]             # assign sheet containing derivative data as obtained from Eagle
shtZ = wb.sheets['Summary']             # assign summary sheet

# create the individual derivative checker files
excel.DisplayAlerts = False             # suppress Excel warning dialogues
for fname in notebook.tqdm(fnames):
    shtH.clear()                                                                     # clear the receiving holdings sheet
    shtH.range('A1').options(index = False).value = wbH[wbH['Entity Name'] == fname] # paste fund holdings
    shtD.clear()                                                                     # clear the receiving deltas sheet    
    shtD.range('A1').options(index = False).value = wbD[wbD['Entity Name'] == fname] # paste fund deltas   
    fndcode          = wbH[wbH['Entity Name'] == fname].iloc[0, 36]                  # get the fund code
    shtZ['A3'].value = 'Not a UT' if twoA.loc[twoA['Fund Code'] == fndcode].iat[0,2] != 'UT' else 'UT'
    wb.save(os.path.join(pthLOCAL,f'{fndcode} Derv Calc {rptDate}.xlsx'))            # save the file
    
# close the derv template workbook
excel.DisplayAlerts = True              # unsuppress Excel warning dialogues
wb.close()

print(f'Creating individual fund derivative calculation files completed: {timediff(start_time, time.time())}', '\n')

Creating individual fund derivative calculation files ...


  0%|          | 0/1 [00:00<?, ?it/s]

Creating individual fund derivative calculation files completed: 9.1sec


In [15]:
# Save the fund derivative cover calc files

start_time = time.time()
print('Saving the fund derivative calculation files to Export folder ...')

# create and save the derivative cover summary file in the Exports folder and then clear the temporary local folder
# copy the locally (C:\) stored derivative calc files to a PIM network folder
for file in notebook.tqdm(os.listdir(pthLOCAL)):    
    shutil.copy(os.path.join(pthLOCAL, file), pthEXPORTS)

print(f'Save the fund derivative calculation files to Export folder completed: {timediff(start_time, time.time())}', '\n')

Saving the fund derivative calculation files to Export folder ...


  0%|          | 0/1 [00:00<?, ?it/s]

Save the fund derivative calculation files to Export folder completed: 0.1sec


In [16]:
# Create and save the derivative cover Summary sheet if required

print(f'Summary sheet required? {summ_yn}')

if summ_yn == 'Yes':
    start_time = time.time()
    print('Populating and then saving the Summary sheet ...')

    wbS            = xw.Book(pthDerv + r'\derv_summary.xlsx')
    shtS           = wbS.sheets['S']                         # derivative cover summary sheet
    fcodes         = pd.DataFrame(wbH['Entity ID'].unique()) # turn frames from numpy.ndarray to a pandas dataframe
    fcodes.columns = ['Entity ID']                           # give the vector of fund codes a column name
    shtS.range(f'A2:Q{len(fcodes)+1000}').clear_contents()   # clear the derivative cover summary sheet below the headings
    #https://stackoverflow.com/questions/66882703/clearing-selected-table-in-xlwings

    shtS.range('A1').options(index = False).value = fcodes   # paste fund holdings onto the calc sheet from the source

    excel.DisplayAlerts = False                              # suppress Excel warning dialogues

    # display date of report
    shtS['A1'].add_hyperlink(pthEXPORTS,'Entity ID')
    shtS['A1'].font.bold = True
    shtS['O1'].add_hyperlink(pthEXPORTS)
    shtS['O1'].value = f'Derivative cover calcs {wbH.iloc[0, 35].strftime("%a %d %b %Y")}'
    shtS['O1'].font.bold = True
    shtS['P1'].add_hyperlink(pthFundCodes,'Team')
    shtS['P1'].font.bold = True
    shtS['Q1'].add_hyperlink(pthOverdrafts,'PIM Overdrafts')
    shtS['Q1'].font.bold = True

    # summarise per fund results
    for index, row in notebook.tqdm(fcodes.iterrows()):
        shtS['A' + str(index + 2)].add_hyperlink(fr'{pthMandates}\{row[0]} Rules.docx',f'{row[0]}')
        shtS['B' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$G$3"
        shtS['C' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$E$1"
        shtS['D' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$C$35"
        shtS['E' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$E$35/100"
        shtS['F' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$C$37"
        shtS['G' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$C$33"
        shtS['H' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$E$6/100"
        shtS['I' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$E$10/100"
        shtS['J' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$E$14/100"
        shtS['K' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$C$42"
        shtS['L' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Summary'!$C$56/100"
        shtS['M' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Glbl Expsr'!$D$2"   
        shtS['N' + str(index + 2)].value = fr"='{pthEXPORTS}\[{row[0]} Derv Calc {rptDate}.xlsx]Glbl Expsr'!$D$3"
        shtS['O' + str(index + 2)].add_hyperlink(fr'{pthEXPORTS}\{row[0]} Derv Calc {rptDate}.xlsx',f'{row[0]} calc sheet')
        shtS['P' + str(index + 2)].value = twoA.loc[twoA['Fund Code'] == f'{row[0]}'].iat[0,1] #investment team lookup
        # Using at[] and iat[] instead of loc[] and iloc[]
        # https://medium.com/codex/dont-use-loc-iloc-with-loops-in-python-instead-use-this-f9243289dde7

    # sort the results
    last_row         = shtS.range(1,1).end('down').row
    first_col_range  = shtS.range(f'D2:D{last_row}')
    second_col_range = shtS.range(f'C2:C{last_row}')
    third_col_range  = shtS.range(f'B2:B{last_row}')
    shtS.range(f'A2:P{last_row}').api.Sort(Key1 = first_col_range.api,  Order1=2, Header=2, Orientation=1) # pct cash cover
    shtS.range(f'A2:P{last_row}').api.Sort(Key1 = second_col_range.api, Order1=2, Header=2, Orientation=1) # qty dervs
    shtS.range(f'A2:P{last_row}').api.Sort(Key1 = third_col_range.api,  Order1=2, Header=2, Orientation=1) # UT or not

    # format the decimals
    # Change column number format - https://stackoverflow.com/questions/66008147/how-to-change-the-number-format-of-the-column-that-a-range-is-using-xlwings-pyh
    # Excel Number Formats          https://python-pptx.readthedocs.io/en/latest/api/enum/ExcelNumFormat.html
    shtS.range(f'D2:N{last_row}').api.EntireColumn.NumberFormat = "0.0000%"


    print('Saving the Summary sheet ...')

    # save all the fund holdings and fund deltas from Eagle on the day's summary sheet as a record for the reporting day
    # set up the summary sheet
    shtH = wbS.sheets['Holdings']  # sheet containing all the fund holdings as obtained from Eagle
    shtH.clear() # clear the receiving holdings sheet
    shtH.range('A1').options(index = False).value = wbH # paste fund holdings onto the calc sheet from the source
    shtD = wbS.sheets['Deltas']    # sheet containing all the fund derivative data as obtained from Eagle
    shtD.clear() # clear the receiving deltas sheet    
    shtD.range('A1').options(index = False).value = wbD # paste deltas onto the calc sheet from the source

    excel.DisplayAlerts = True     # unsuppress Excel warning dialogues

    # save and then close the summary file
    wbS.save(os.path.join(pthEXPORTS,f'Derv {rptDate}.xlsx'))
    wbS.close()

    # delete contents of the temporary local folder
    for filename in notebook.tqdm(os.listdir(pthLOCAL)):    
        file_path = os.path.join(pthLOCAL, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Couldn't delete {file_path} because {e}")

    print(f'Populating and then saving the Summary sheet completed: {timediff(start_time, time.time())}', '\n')

Summary sheet required? No


In [1]:
# Open the derivative cover summary file for review

if summ_yn == 'Yes':
    start_time = time.time()
    print('Opening the Summary sheet for review ...')
    excel.Workbooks.Open(pthEXPORTS + f'/Derv {rptDate}.xlsx')
    print(f'Opening the Summary sheet for review completed: {timediff(start_time, time.time())}')
    
print(f'Roundtrip time for deivative calcs and summary sheet: {timediff(start_time_overlord, time.time())}', '\n')

NameError: name 'time' is not defined